In [2]:
    import pandas as pd
    import numpy as np
    import time
    from datetime import datetime, timedelta
    from pandas.tseries.offsets import MonthEnd
    from elexon import ElexonRawClient

In [3]:
class PriceDataAccesser():
    
    '''This class takes in a user's api_key = api_key, start = start date in YYYY-mm-dd, 
    end = end date in YYYY-mm-dd and Elexon data code in XXXXXX. Generated data will have 30 min increments.
    Suggest using the 1st day of the month in string format.
    
    Methods include: 
    
    .date_ranger(self)
    .price_getter(self)
    .dataframe_maker(self)
    
    Need to call .price_getter before .dataframe_maker
    
    by K O'Donnell'''
    
    
    global df
    
    def __init__(self, api_key, start, end, elexon_code):
        
        self.api_key = api_key
        self.start = start
        self.end = end
        self.elexon_code = elexon_code
    
    def date_ranger(self):
        
        '''This method simply generates a the list of dates within user defined start and end'''
        
        for beg in pd.date_range(self.start, self.end, freq='MS'):
            print(beg.strftime("%Y-%m-%d"), (beg + MonthEnd(1)).strftime("%Y-%m-%d"))
    
    def price_getter(self):
        
        '''This method pulls data from Exelon. Need to use api key etc as defined previously'''
        self.prices_since_date = []
        self.api = ElexonRawClient(self.api_key) 
        # Put in start of and end of motnh (i.e 31st etc)
        for date in pd.date_range(self.start, self.end, freq='MS'):
            date = self.api.request(self.elexon_code,FromSettlementDate = date.strftime("%Y-%m-%d"), 
                                         ToSettlementDate = (date + MonthEnd(1)).strftime("%Y-%m-%d"))
            self.prices_since_date.append(date)
            
    def dataframe_maker(self):
        
        '''This method changes the pulled data into a Pandas Dataframe, adds datetimeindex, 
        removes settlemnt date as it is redundant at this stage. 
        Example usage: desired_dataframe = dataaccesser.dataframe_maker()'''
        
        for i in range(1,len(self.prices_since_date)):
            s = self.prices_since_date[0]
            s += self.prices_since_date[i]
            self.df = pd.DataFrame(s)
            self.df = self.df.drop('settlementDate', axis = 1)
            self.dates = pd.date_range(start=self.start,periods=len(self.df), freq='30T') 
            self.df.index = self.dates
            
        return self.df

In [4]:
start = '2020-01-01'
end = '2020-02-01'
data_access = PriceDataAccesser('k62su1yu4pcft7j', start, end, elexon_code = 'DERSYSDATA')

In [5]:
data_access.price_getter()
data_access.dataframe_maker()

,recordType,settlementPeriod,systemSellPrice,systemBuyPrice,bSADDefault,priceDerivationCode,reserveScarcityPrice,indicativeNetImbalanceVolume,sellPriceAdjustment,buyPriceAdjustment,...,totalSystemAcceptedBidVolume,totalSystemTaggedAcceptedOfferVolume,totalSystemTaggedAcceptedBidVolume,totalSystemAdjustmentSellVolume,totalSystemAdjustmentBuyVolume,totalSystemTaggedAdjustmentSellVolume,totalSystemTaggedAdjustmentBuyVolume,activeFlag,replacementPrice,replacementPriceCalculationVolume
2020-01-01 00:00:00,SSB,1,50.90000,50.90000,F,P,NULL,54.3365,0.0,0.0,...,-449.397,840.233,-449.397,-455.0,117.5,-455.0,117.5,True,NaN,NaN
2020-01-01 00:30:00,SSB,2,51.00000,51.00000,F,P,NULL,194.7133,0.0,0.0,...,-330.283,861.442,-330.283,-455.0,117.5,-455.0,117.5,True,NaN,NaN
2020-01-01 01:00:00,SSB,3,29.37006,29.37006,F,N,NULL,-71.4292,0.0,0.0,...,-566.429,832.500,-566.429,-455.0,117.5,-454.0,117.5,True,NaN,NaN
2020-01-01 01:30:00,SSB,4,28.52000,28.52000,F,N,NULL,-122.2322,0.0,0.0,...,-670.263,885.555,-670.263,-455.0,117.5,-454.0,117.5,True,NaN,NaN
2020-01-01 02:00:00,SSB,5,25.78000,25.78000,F,N,NULL,-90.9042,0.0,0.0,...,-584.793,831.389,-584.793,-455.0,117.5,-454.0,117.5,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 21:30:00,SSB,44,48.00000,48.00000,F,P,NULL,283.0102,0.0,0.0,...,-461.806,1504.116,-461.806,-760.3,0.0,-760.3,0.0,True,NaN,NaN
2020-02-29 22:00:00,SSB,45,45.00000,45.00000,F,P,NULL,285.9822,0.0,0.0,...,-92.663,1501.528,-92.663,-1123.8,0.0,-1123.8,0.0,True,NaN,NaN
2020-02-29 22:30:00,SSB,46,43.00000,43.00000,F,P,NULL,52.8332,0.0,0.0,...,-44.050,1219.683,-44.050,-1123.8,0.0,-1123.8,0.0,True,NaN,NaN
2020-02-29 23:00:00,SSB,47,12.66000,12.66000,F,N,NULL,-35.7840,0.0,0.0,...,-125.984,786.500,-125.984,-1123.8,427.5,-1122.8,427.5,True,NaN,NaN


In [36]:
class DemandDataAccesser():
    
    '''This class takes in a user's api_key = api_key, start = start date in YYYY-mm-dd, 
    end = end date in YYYY-mm-dd and Elexon data code in XXXXXX. Generated data will have 30 min increments.
    Suggest using the 1st day of the month in string format.
    
    Methods include: 
    
    .date_ranger(self)
    .demand_getter(self)
    .dataframe_maker(self)
    
    Need to call .demand_getter before .dataframe_maker. Use 'SYSDEM' Elexon code for demand.
    
    Output is a tuple: choose[0] for ITSDO and choose[1] for TSDF. Other details can be removed using normal Pandas
    
    by K O'Donnell'''
    
    
    global df_dem, df_ITSDO, df_TSDF 
    
    def __init__(self, api_key, start, end, elexon_code):
        
        self.api_key = api_key
        self.start = start
        self.end = end
        self.elexon_code = elexon_code
    
    def date_ranger(self):
        
        '''This method simply generates a the list of dates within user defined start and end'''
        
        for beg in pd.date_range(self.start, self.end, freq='MS'):
            print(beg.strftime("%Y-%m-%d"), (beg + MonthEnd(1)).strftime("%Y-%m-%d"))
    
    def demand_getter(self):
        
        '''This method pulls data from Exelon. Need to use api key etc as defined previously'''
        self.prices_since_date = []
        self.api = ElexonRawClient(self.api_key) 
        # Put in start of and end of motnh (i.e 31st etc)
        for date in pd.date_range(self.start, self.end, freq='MS'):
            date = self.api.request(self.elexon_code,FromDate = date.strftime("%Y-%m-%d"), 
                                         ToDate = (date + MonthEnd(1)).strftime("%Y-%m-%d"))
            self.prices_since_date.append(date)
            
    def dataframe_maker(self):
        
        '''This method changes the pulled data into a Pandas Dataframe, adds datetimeindex, 
        removes settlemnt date as it is redundant at this stage. 
        Example usage: desired_dataframe = dataaccesser.dataframe_maker()'''
        
        for i in range(1,len(self.prices_since_date)):
            s = self.prices_since_date[0]
            s += self.prices_since_date[i]
            self.df_dem = pd.DataFrame(s)
            self.df_ITSDO = self.df_dem[self.df_dem['recordType']=='ITSDO']
            self.df_TSDF = self.df_dem[self.df_dem['recordType']=='TSDF']
            self.df = self.df_dem.drop('startTimeOfHalfHrPeriod', axis = 1)
            self.dates_ITSDO = pd.date_range(start=self.start,periods=len(self.df_ITSDO), freq='30T')
            self.dates_TSDF = pd.date_range(start=self.start,periods=len(self.df_TSDF), freq='30T') 
            self.df_ITSDO.index = self.dates_ITSDO
            self.df_TSDF.index = self.dates_TSDF
            self.df_ITSDO = self.df_ITSDO.drop('startTimeOfHalfHrPeriod', axis = 1)
            self.df_TSDF = self.df_TSDF.drop('startTimeOfHalfHrPeriod', axis = 1)
            
        return self.df_ITSDO, self.df_TSDF, self.df_dem

In [37]:
data_access = DemandDataAccesser('k62su1yu4pcft7j', start, end, elexon_code = 'SYSDEM')

In [38]:
data_access.demand_getter()
eff =data_access.dataframe_maker()

In [40]:
type(eff)

tuple

In [41]:
eff[0]

,recordType,settlementPeriod,demand,activeFlag
2020-01-01 00:00:00,ITSDO,1,27073.0,True
2020-01-01 00:30:00,ITSDO,2,27612.0,True
2020-01-01 01:00:00,ITSDO,3,27111.0,True
2020-01-01 01:30:00,ITSDO,4,26311.0,True
2020-01-01 02:00:00,ITSDO,5,25693.0,True
...,...,...,...,...
2020-02-29 21:30:00,ITSDO,44,30166.0,True
2020-02-29 22:00:00,ITSDO,45,28872.0,True
2020-02-29 22:30:00,ITSDO,46,27654.0,True
2020-02-29 23:00:00,ITSDO,47,26291.0,True


In [39]:
eff[1]

,recordType,settlementPeriod,demand,activeFlag
2020-01-01 00:00:00,TSDF,1,27395.0,True
2020-01-01 00:30:00,TSDF,2,28042.0,True
2020-01-01 01:00:00,TSDF,3,27890.0,True
2020-01-01 01:30:00,TSDF,4,27009.0,True
2020-01-01 02:00:00,TSDF,5,26332.0,True
...,...,...,...,...
2020-02-29 21:30:00,TSDF,44,29720.0,True
2020-02-29 22:00:00,TSDF,45,28373.0,True
2020-02-29 22:30:00,TSDF,46,27070.0,True
2020-02-29 23:00:00,TSDF,47,25844.0,True
